<a href="https://colab.research.google.com/github/Mudassir-beep/AI-Disease-diagnostics/blob/main/RAG-Faiss-Tiny-Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# ✅ Install necessary packages
!pip install -q streamlit pyngrok pandas torch transformers langchain sentence-transformers


In [36]:

!pip install -q \
    streamlit pyngrok pandas torch transformers \
    sentence-transformers langchain langchain-community faiss-cpu




In [37]:
!pip install -q streamlit pyngrok pandas torch transformers sentence-transformers langchain langchain-community


In [38]:
%%writefile app.py

import streamlit as st
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.schema import Document

st.set_page_config(page_title="TinyLlama RAG Chatbot", layout="wide")
st.title("🧠 TinyLlama RAG Chatbot")

# ✅ Path to uploaded CSV file
file_path = "/content/gaza_genocide_docs_clean.csv"

# ✅ Load CSV
try:
    df = pd.read_csv(file_path)
    if 'text' not in df.columns:
        st.error("❌ CSV must contain a column named 'text'.")
        st.stop()
    st.success(f"✅ Loaded {len(df)} rows from: {file_path}")
except Exception as e:
    st.error(f"❌ Failed to load CSV: {e}")
    st.stop()

# ✅ Convert to LangChain Documents
try:
    documents = [Document(page_content=str(row)) for row in df['text'].dropna()]
    if not documents:
        st.error("❌ No text found in 'text' column.")
        st.stop()
except Exception as e:
    st.error(f"❌ Error converting DataFrame to documents: {e}")
    st.stop()

# ✅ Load TinyLlama Model on GPU (if available)
with st.spinner("⏳ Loading TinyLlama..."):
    try:
        model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )
        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
        llm = HuggingFacePipeline(pipeline=pipe)
    except Exception as e:
        st.error(f"❌ Model loading failed: {e}")
        st.stop()

# ✅ Build FAISS Vector DB using GPU
with st.spinner("🔍 Building FAISS Vector DB..."):
    try:
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        db = FAISS.from_documents(documents, embedding=embeddings)
        retriever = db.as_retriever()
        qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    except Exception as e:
        st.error(f"❌ FAISS or RAG failed: {e}")
        st.stop()

# ✅ UI for asking question
user_question = st.text_input("💬 Ask a question:")
if user_question:
    with st.spinner("🤖 Generating answer..."):
        try:
            response = qa_chain.run(user_question)
            st.markdown("### ✅ Answer:")
            st.write(response)
        except Exception as e:
            st.error(f"❌ Answer generation failed: {e}")


Overwriting app.py


In [39]:
!ngrok config add-authtoken 30Hp3ggHO2VyEkOtL7XXIrnzgBv_2dhqUQVo6XVQzzgpt5McZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [40]:
!pkill -f streamlit
!pkill -f ngrok
# ✅ Launch Streamlit via ngrok in Colab
from pyngrok import ngrok
import threading
import time

def run_app():
    !streamlit run app.py &

thread = threading.Thread(target=run_app)
thread.start()

# Wait for Streamlit to spin up
time.sleep(5)
public_url = ngrok.connect(8501)
print("🔗 Public URL:", public_url)



  Stopping...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.168.242.86:8501

🔗 Public URL: NgrokTunnel: "https://e39c1d09d4e5.ngrok-free.app" -> "http://localhost:8501"
